# Convex.jl - A native Julia package for real or complex domain Convex Optimization

## Ayush Pandey | PyData Delhi 2017

[https://github.com/Ayush-iitkgp/PyData2017Presentation](https://github.com/Ayush-iitkgp/PyData2017Presentation)

# About Me 

* **BS and MS in Mathematics and Computing Sciences (July'12-June'17) at Indian Institute of Technology Kharagpur **

* **Google Summer of Code 2016 and 2017 student under the Julia Language**

* **GitHub:** [Ayush-iitkgp](https://github.com/ayush-iitkgp)

* **Website:** [https://ayush-iitkgp.github.io](https://ayush-iitkgp.github.io/) 



<!----#### Blogging my GSoC'16 experience at [http://ayush-iitkgp.rhcloud.com](http://ayush-iitkgp.rhcloud.com/) ---->

<!--- ## CVX.jl team

* [CVX.jl](https://github.com/cvxgrp/CVX.jl): Madeleine Udell, Karanveer Mohan, David Zeng, Jenny Hong
<!---* [ParallelSparseMatMul.jl](https://github.com/madeleineudell/ParallelSparseMatMul.jl): Madeleine Udell
--->

# Outline

### Introduction to Julia Programming Language
* Why is Julia faster than Python?

### Convex Optimization

### Why Convex.jl?

### Tutorial

### Use Cases

* Portfolio Optimization
* Logistic Regression
* Fidelity in Quantum Information

### Benchmarking
<!--* Benchmark -->

# Introduction to Julia Programming Language

* Free and open source (MIT licensed) programming language for high performance scientific computing.

* Very much pythonic in syntax.

* Deploys [just-in-time (JIT)](https://en.wikipedia.org/wiki/Just-in-time_compilation) compilation, implemented using [LLVM](https://en.wikipedia.org/wiki/LLVM)

<!-- just-in-time (JIT) compilation is compilation done during execution of a program – at run time -->

* Arrays are 1-indexed.

* Great Julia [tutorials](https://www.youtube.com/watch?v=vWkgEddb4-A) at SciPy 2014 by David Sanders.

## Why is Julia faster than Python?

In [ ]:
a = 1
b = 2
c = a + b

### Python Addition

* Here the python interpreter knows only that 1 and 2 are objects, but not what type of object they are.

#### Step 1: Assign 1 to a

> **1a.** Set a->PyObject_HEAD->typecode to integer


> **1b.** Set a->val = 1

#### Step 2: Assign 2 to b

> **2a.** Set b->PyObject_HEAD->typecode to integer


> **2b.** Set b->val = 2

#### Step 3: call binary_add(a, b)

> **3a.** find typecode in a->PyObject_HEAD

> **3b.** a is an integer; value is a->val

> **3c.** find typecode in b->PyObject_HEAD

> **3d.** b is an integer; value is b->val

> **3e.** call binary_add(int, int)(a->val, b->val)

> **3f.** result of this is result, and is an integer.

#### Step 4: Create a Python object c

> **4a.** set c->PyObject_HEAD->typecode to integer

> **4b.** set c->val to result

### Julia Addition

* Julia's JIT compilation helps to infer the types of the variable at run-time.

#### Step 1: Assign 1 to a

> **1a.** At complie time, type of a is set to **Integer**


> **1b.** Set value of a to 1.

#### Step 2: Assign 2 to b

> **2a.** At complie time, type of b is set to **Integer**


> **2b.** Set value of b to 2.

#### Step 3: call binary_add(a, b)

* Since types of a and b are known. 

> **3a.** call binary_add(int, int)(a, b)

> **3b.** result of this is an integer.

#### Step 4: Create an Int c

> **4a.** set c to result of binary_add(Int, Int)

# Convex Optimization

### What is Convex function? 
$f$ is **convex** if for all $\theta \in [0,1]$
$$
f(\theta x + (1-\theta)y ) \leq \theta f(x) + (1-\theta) f(y)
$$

equivalently, 

* $f$ has nonnegative (upward) curvature
* $f'' \geq 0$ (if $f$ is differentiable)
* Geometrically, the line joining any 2 points on $f$ always lies above the graph of $f$

<!---* sublevel sets $\{x: f(x) \leq \alpha\}$ are convex sets --->

![chords](chord.png)

**Note - A function is called affine iff it is both convex and concave.**

## What is Convex optimization ?

### Functional Form

$$
\begin{array}{ll} 
\mbox{minimize}  & f_0(x) \\
\mbox{subject to} & f_i(x) \leq 0, \quad i=1, \ldots, m_1\\
& h_j(x) = 0, \quad j=1, \ldots, m_2\\
\end{array}
$$

* variable $x\in \mathbf{R}^n$
* $f_i$ are all convex
* $h_j$ are all affine

**Note 1- If $x\in \mathbf{R}^n$ then we refer the problem as real-domain convex optimization and if $x\in \mathbf{C}^n$ then we refer it as complex-domain optimization problem. Also if the coefficients of the variablex are complex numbers then it is complex-doamin optimization problem.** 

**Note 2- It is important to note that if $x\in \mathbf{C}^n$ then we no longer have inequality constraint.**

## Why do we need another package if we have available solvers like SCS, Mosek?

### Convex optimization (conic form)

$$
\begin{array}{ll} 
\mbox{minimize}  & c^T x \\
    \mbox{subject to} & Ax = b\\
    & x \in \mathcal K\\
\end{array}
$$

where $\mathcal K$ is a **convex cone**

$$ x \in \mathcal K \iff rx \in \mathcal K, \quad \forall r>0$$

examples:

* positive orthant 

$$\mathcal K_+ = \{x: x\geq 0\}$$
    
* second order cone 

$$\mathcal K_{\mathrm{SOC}} = \{(x,t): \|x\|_2 \leq t\}$$
    
* semidefinite cone 

$$\mathcal K_{\mathrm{SDP}} = \{X: X = X^T,~ v^T X v \geq 0,~ \forall v \in \mathbf{R}^n\}$$

**Does the previous slide look complicated? It turns out that the above solvers only understand the conic form which is hard for us to write and this is the reason why we need a package for convex optimization where we can express our problem in functional form and let the package manage the difficult part of converting the problem to conic form.**

# Quick Tutorial

In [3]:
using Convex

## Variables

In [4]:
# Scalar variable
x = Variable()
# (Column) vector variable
y = Variable(4)
# Matrix variable
z = Variable(4, 2)

Variable of
size: (4, 2)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

# Expressions

* We can operate on variables to form *convex expressions*

In [5]:
# indexing, multiplication, addition
e1 = y[1] + 2*x

# expressions can be affine, convex, or concave
e3 = sqrt(x)

AbstractExpr with
head: geomean
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConcaveVexity()


# Constraints

In [6]:
# affine equality constraint
A = randn(3,4); b = randn(3)
constraint1 = A*y == b

Constraint:
== constraint
lhs: AbstractExpr with
head: *
size: (3, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

rhs: [-1.6048077944321295,2.1044876445367,0.6535176954608789]
vexity: Convex.AffineVexity()

In [8]:
# convex inequality constraint
constraint2 = norm(y,2) <= 2

Constraint:
<= constraint
lhs: AbstractExpr with
head: norm2
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()

rhs: 2
vexity: Convex.ConvexVexity()

# Problems

* Now let's combine the above 3 steps to define our problem

In [9]:
objective = 2*x + 1 - sqrt(sum(y))
p = minimize(objective, constraint1, constraint2)

Problem:
minimize AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

subject to
Constraint:
== constraint
lhs: AbstractExpr with
head: *
size: (3, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

rhs: [-1.6048077944321295,2.1044876445367,0.6535176954608789]
vexity: Convex.AffineVexity()
		Constraint:
<= constraint
lhs: AbstractExpr with
head: norm2
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()

rhs: 2
vexity: Convex.ConvexVexity()
current status: not yet solved

In [10]:
# solve the problem
solve!(p)
p

----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 34
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 8, constraints m = 15
Cones:	primal zero / dual free vars: 4
	linear vars: 3
	soc vars: 8, soc blks: 2
Setup time: 8.36e-05s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  2.49e-05 
    20|      inf       inf      -nan      -inf      -inf       inf  5.35e-05 
----------------------------------------------------------------------------
Status: Unbounded
Timing: Solve tim

Problem:
minimize AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

subject to
Constraint:
== constraint
lhs: AbstractExpr with
head: *
size: (3, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

rhs: [-1.6048077944321295,2.1044876445367,0.6535176954608789]
vexity: Convex.AffineVexity()
		Constraint:
<= constraint
lhs: AbstractExpr with
head: norm2
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()

rhs: 2
vexity: Convex.ConvexVexity()
current status: Unbounded

In [11]:
p.optval

-0.9999999999999999

In [12]:
x.value

-0.49999943073238445

# Why do we need complex-domain optimization convex package?

## Applications of complex-domain convex optimization

1. **Phase retreival from sparse signals** used in MRI Imaging of the Brain

**Mathematical Formulation**

*find x*

*satisfying A(x) = A($x_0$) = b*

*where $x_0$ in $x\in \mathbf{C}^n$*

*A($x_0$) = {|$<a_k, x_0>$|^2 : k = 1, 2, . . . , m }*

![MRI Imaging](Brain_MRI.jpg)


**Used in demodulation of the mutually interfering digital streams of information** that occur in areas such as wireless communications, high-speed data transmission, DSL, satellite communication, digital television, and magnetic recording. 

Here the problem is detecting interference between multiple users in a wireless communication network.

![Wireless Communication](wirelesscommunication.jpg)

**Optimization in power grids**: The problem of finding the most efficient dispatch of generators in a power grid subject to meeting the demand for power and respecting network constraints. Here the complex variable x represents steady-state voltages in an AC power network.

Here one of the variables which represents steady-state voltages in an AC power network is a complex variable so it again becomes complex-domain optimization problem.


![Power Grids](powergrid.jpg)

# How Convex.jl handles the complex-domain optimization problem?

## Things to keep in mind

* The objective function must be real
* The inequality constraints must be real
* The last entry of the Second Order Conic Constraint must be real.
* All variables in exponential cones must be real.

In [ ]:
A complex equality constraint could be transfored to corresponding 2 equality constraints.

# My Work

I have started with implemeting the support for complex coefficients which means we keep the variable (say x) as real but the coefficients as complex. 

In [2]:
# Creating a real variable
using Convex
x = Variable()

Variable of
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

In [3]:
# Creating an objective function
objective = 2*x

AbstractExpr with
head: *
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()


In [4]:
# Creating complex equality constraints
c1 = (2+3im)*x == (6+9im)
c2 = x<=5

Constraint:
<= constraint
lhs: Variable of
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()
rhs: 5
vexity: Convex.AffineVexity()

In [5]:
p = minimize(objective, c1, c2)

Problem:
minimize AbstractExpr with
head: *
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

subject to
Constraint:
== constraint
lhs: AbstractExpr with
head: *
size: (1, 1)
sign: Convex.ComplexSign()
vexity: Convex.AffineVexity()

rhs: 6 + 9im
vexity: Convex.AffineVexity()
		Constraint:
<= constraint
lhs: Variable of
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()
rhs: 5
vexity: Convex.AffineVexity()
current status: not yet solved

In [9]:
solve!(p,verbose=false)
x.value
p.optval

# Solution
# x.value = 3.0001651341687965
# p.optval = 6.000269523275286



----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 5
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 2, constraints m = 4
Cones:	primal zero / dual free vars: 3
	linear vars: 1
Setup time: 5.35e-05s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan       inf      -inf       inf  2.26e-05 
    60| 7.69e-06  5.72e-05  3.14e-05  6.00e+00  6.00e+00  9.46e-17  1.02e-04 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 1.04e-04s
	Lin-sys: nnz in L

5.999939822156568

# Future work

* Implementing complex variables
* Implemeting Complex SemiDefinite Programming

# Questions